# Census Disability Demographics

**Author:** Luke Steuber  
**Last Updated:** 2026-02-13

Analysis of Census Bureau disability data covering 44.1 million Americans (13.4%) across counties, states, demographic groups, and trends from 2010-2023.

In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Datasets

In [ ]:
# County-level data
df_county = pd.read_csv('../census_disability_by_county_2022.csv')
print(f"County data: {df_county.shape[0]} counties")
df_county.head()

In [ ]:
# State-level data
df_states = pd.read_csv('../census_disability_states_2021.csv')
print(f"State data: {df_states.shape[0]} states/territories")
df_states.head()

In [ ]:
# National characteristics
with open('../census_disability_characteristics_2022.json') as f:
    characteristics = json.load(f)
print(f"Characteristics keys: {list(characteristics.keys())}")
characteristics['overall']

In [ ]:
# Disability by race
with open('../census_disability_by_race_2022.json') as f:
    race_data = json.load(f)
df_race = pd.DataFrame(race_data)
print(f"Race/ethnicity groups: {df_race.shape[0]}")
df_race

In [ ]:
# Disability by age and sex
with open('../census_disability_by_age_sex_2022.json') as f:
    age_sex_data = json.load(f)
print(f"Age/sex breakdown: {len(age_sex_data)} age groups")
age_sex_data[0]

In [ ]:
# 5-year national trends
with open('../census_disability_national_trends.json') as f:
    trends_5yr = json.load(f)
print(f"5-year trend: {trends_5yr['year_range']}")
trends_5yr

In [ ]:
# 13-year clean trends
with open('../census_disability_trends_clean.json') as f:
    trends_13yr = json.load(f)
df_trends = pd.DataFrame(trends_13yr['annual_trends'])
print(f"13-year trend: {df_trends.shape[0]} years ({df_trends['year'].min()}-{df_trends['year'].max()})")
df_trends.head()

## 2. Key Findings Summary

In [ ]:
print("=" * 60)
print("KEY FINDINGS: Census Disability Data 2010-2023")
print("=" * 60)
print(f"\n📊 Current Snapshot (2022-2023):")
print(f"   • 44.1 million Americans with disabilities (13.4%)")
print(f"   • Disability rate rose from 11.9% (2010) to 13.6% (2023)")
print(f"\n🔍 By Disability Type (most to least common):")
if 'by_disability_type' in characteristics:
    types = characteristics['by_disability_type']
    sorted_types = sorted(types.items(), key=lambda x: x[1]['rate_pct'], reverse=True)
    for dtype, data in sorted_types:
        print(f"   • {dtype.replace('_', ' ').title()}: {data['rate_pct']:.1f}%")
print(f"\n📈 13-Year Trend:")
print(f"   • 2010: {df_trends.iloc[0]['disability_rate_pct']:.1f}%")
print(f"   • 2023: {df_trends.iloc[-1]['disability_rate_pct']:.1f}%")
print(f"   • Absolute increase: {df_trends.iloc[-1]['disability_rate_pct'] - df_trends.iloc[0]['disability_rate_pct']:.1f} percentage points")
print("=" * 60)

## 3. Visualizations

### 3a. 13-Year Disability Rate Trend (2010-2023)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(df_trends['year'], df_trends['disability_rate_pct'], marker='o', linewidth=2.5, markersize=8, color='#2E86AB')
ax.fill_between(df_trends['year'], df_trends['disability_rate_pct'], alpha=0.2, color='#2E86AB')
ax.set_xlabel('Year', fontsize=12, fontweight='bold')
ax.set_ylabel('Disability Rate (%)', fontsize=12, fontweight='bold')
ax.set_title('U.S. Disability Rate 2010-2023: Rising from 11.9% to 13.6%', fontsize=14, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3)
ax.set_ylim(11, 14)
for i, row in df_trends.iterrows():
    if i % 2 == 0:
        ax.annotate(f"{row['disability_rate_pct']:.1f}%", 
                   xy=(row['year'], row['disability_rate_pct']), 
                   xytext=(0, 10), 
                   textcoords='offset points',
                   fontsize=9,
                   ha='center')
plt.tight_layout()
plt.show()

### 3b. Disability Rates by Type

In [ ]:
if 'by_disability_type' in characteristics:
    types_data = characteristics['by_disability_type']
    df_types = pd.DataFrame([
        {'type': k.replace('_', ' ').title(), 'rate': v['rate_pct']} 
        for k, v in types_data.items()
    ]).sort_values('rate', ascending=False)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    colors = sns.color_palette('viridis', len(df_types))
    bars = ax.barh(df_types['type'], df_types['rate'], color=colors, edgecolor='black', linewidth=0.5)
    ax.set_xlabel('Disability Rate (%)', fontsize=12, fontweight='bold')
    ax.set_title('Disability Prevalence by Type (2022)', fontsize=14, fontweight='bold', pad=20)
    ax.grid(axis='x', alpha=0.3)
    for i, (bar, row) in enumerate(zip(bars, df_types.itertuples())):
        ax.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2, 
               f"{row.rate:.1f}%", va='center', fontsize=10, fontweight='bold')
    plt.tight_layout()
    plt.show()

### 3c. Top 20 Counties by Disability Rate

In [ ]:
top20 = df_county.nlargest(20, 'disability_rate').copy()
top20['label'] = top20.apply(lambda x: f"{x['county_name'].split(',')[0]}, {x['county_name'].split(',')[1].strip()}", axis=1)

fig, ax = plt.subplots(figsize=(12, 8))
colors = sns.color_palette('rocket', len(top20))
bars = ax.barh(range(len(top20)), top20['disability_rate'], color=colors, edgecolor='black', linewidth=0.5)
ax.set_yticks(range(len(top20)))
ax.set_yticklabels(top20['label'], fontsize=10)
ax.set_xlabel('Disability Rate (%)', fontsize=12, fontweight='bold')
ax.set_title('Top 20 U.S. Counties by Disability Rate (2022)', fontsize=14, fontweight='bold', pad=20)
ax.grid(axis='x', alpha=0.3)
ax.invert_yaxis()
for i, (bar, rate) in enumerate(zip(bars, top20['disability_rate'])):
    ax.text(bar.get_width() + 0.3, bar.get_y() + bar.get_height()/2, 
           f"{rate:.1f}%", va='center', fontsize=9, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\nHighest disability rate: {top20.iloc[0]['county_name']} ({top20.iloc[0]['disability_rate']:.1f}%)")
print(f"National average: {df_county['disability_rate'].mean():.1f}%")

### 3d. Disability Rates by Race/Ethnicity

In [ ]:
df_race_sorted = df_race.sort_values('disability_rate_pct', ascending=False)

fig, ax = plt.subplots(figsize=(12, 7))
colors = sns.color_palette('mako', len(df_race_sorted))
bars = ax.bar(range(len(df_race_sorted)), df_race_sorted['disability_rate_pct'], 
              color=colors, edgecolor='black', linewidth=0.5)
ax.set_xticks(range(len(df_race_sorted)))
ax.set_xticklabels(df_race_sorted['race_ethnicity'], rotation=45, ha='right', fontsize=10)
ax.set_ylabel('Disability Rate (%)', fontsize=12, fontweight='bold')
ax.set_title('Disability Rates by Race/Ethnicity (2022)', fontsize=14, fontweight='bold', pad=20)
ax.grid(axis='y', alpha=0.3)
ax.axhline(df_race[df_race['race_ethnicity'] == 'Total']['disability_rate_pct'].values[0], 
          color='red', linestyle='--', linewidth=2, alpha=0.7, label='National Average')
for bar, rate in zip(bars, df_race_sorted['disability_rate_pct']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2, 
           f"{rate:.1f}%", ha='center', fontsize=9, fontweight='bold')
ax.legend(loc='upper right', fontsize=10)
plt.tight_layout()
plt.show()

## 4. Geographic Distribution

In [ ]:
state_summary = df_county.groupby('county_name').apply(
    lambda x: x['county_name'].iloc[0].split(', ')[1] if ', ' in x['county_name'].iloc[0] else 'Unknown'
).value_counts().head(10)

print("\nCounties per state (top 10):")
print(state_summary)

In [ ]:
print("\nNational disability statistics:")
print(f"Total counties analyzed: {df_county.shape[0]:,}")
print(f"Mean disability rate: {df_county['disability_rate'].mean():.2f}%")
print(f"Median disability rate: {df_county['disability_rate'].median():.2f}%")
print(f"Range: {df_county['disability_rate'].min():.2f}% to {df_county['disability_rate'].max():.2f}%")
print(f"Standard deviation: {df_county['disability_rate'].std():.2f}%")

## 5. Disability Type Breakdown

In [ ]:
type_cols = ['hearing_rate', 'vision_rate', 'cognitive_rate', 'ambulatory_rate', 'self_care_rate', 'independent_living_rate']
type_means = df_county[type_cols].mean().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(12, 6))
colors = sns.color_palette('coolwarm', len(type_means))
bars = ax.bar(range(len(type_means)), type_means.values, color=colors, edgecolor='black', linewidth=0.5)
ax.set_xticks(range(len(type_means)))
labels = [label.replace('_rate', '').replace('_', ' ').title() for label in type_means.index]
ax.set_xticklabels(labels, rotation=45, ha='right', fontsize=11)
ax.set_ylabel('Mean Rate Across Counties (%)', fontsize=12, fontweight='bold')
ax.set_title('Average Disability Rates by Type Across U.S. Counties (2022)', fontsize=14, fontweight='bold', pad=20)
ax.grid(axis='y', alpha=0.3)
for bar, rate in zip(bars, type_means.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
           f"{rate:.2f}%", ha='center', fontsize=10, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nMost common disability type:", labels[0], f"({type_means.values[0]:.2f}%)")

## 6. Summary

This analysis reveals:

1. **Rising trend**: Disability prevalence increased from 11.9% (2010) to 13.6% (2023), affecting 44.1 million Americans.
2. **Ambulatory disabilities** are most common, followed by cognitive and independent living difficulties.
3. **Geographic variation**: County-level disability rates range from less than 5% to over 30%, with clusters in Appalachia and the Deep South.
4. **Demographic disparities**: Significant variation across race/ethnicity groups, with some populations experiencing rates 30-40% above the national average.
5. **Age correlation**: Disability rates increase sharply with age, particularly after 65.

These datasets enable detailed analysis of disability demographics for policy research, accessibility planning, and resource allocation.

**Data Sources**: U.S. Census Bureau ACS (2010-2023), Tables S1810 and B18101